# Supervised Classification with Word2Vec

We use the document vectors derived from Word2Vec to classify newspaper articles according to the type of crime they report. In this notebook, we use supervised algorithms to classify.

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier

In [ ]:
configuration = 1    # possible values are: 1, 2, 3
mean = 'tfidf'

###
# configuration=1 --> P1: tokenization, stop word removal
# configuration=2 --> P2: tokenization, stop word removal, lemmatization
# configuration=3 --> P3: tokenization, stop word removal, lemmatization, keyphrase extraction
###

folder="configuration_"+str(configuration)

if configuration == 1:
    lemmatization=False
    bigram=False
elif configuration == 2:
    lemmatization=True
    bigram=False
elif configuration == 3:
    lemmatization=True
    bigram=True

Load the .csv file resulted from the notebook named "feature_extraction.ipynb"

In [ ]:
vectors = pd.read_csv(folder+"/"+mean+"/csv/retrained_vectors.csv")

In [ ]:
vectors

In [ ]:
vectors = vectors.reset_index(drop=True)

In [ ]:
Y = vectors['target']
Y

In [ ]:
label_to_index = {v: i for i, v in enumerate(dict(Counter(Y)).keys())}
label_to_index

In [ ]:
index_to_label = {i: v  for i, v in enumerate(dict(Counter(Y)).keys())}
index_to_label

In [ ]:
Y = Y.map(label_to_index)

In [ ]:
Y = Y.values
Y

In [ ]:
X = vectors.drop(columns=['url', 'title', 'newspaper', 'text', 'date', 'time', 'preprocessed', 'target']).values

In [ ]:
X.shape

We split the dataset into training set and test set

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.340, random_state=42)

In [ ]:
X_train.shape

In [ ]:
Y_train

In [ ]:
Y_test

In [ ]:
Y = np.vectorize(index_to_label.get)(Y_train)

In [ ]:
Counter(Y)

In [ ]:
Counter(Y_train)

In [ ]:
Counter(Y_test)

# 1. Random Forest

In [ ]:
%%time

forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(X_train, Y_train)

In [ ]:
result = forest.predict(X_test)

In [ ]:
result

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
prediction = Counter(np.equal(result, Y_test))
prediction[True] / (prediction[True] + prediction[False])

# 2. ExtraTreesClassifier

In [ ]:
%%time

etc = ExtraTreesClassifier(n_estimators=200)
etc = etc.fit(X_train, Y_train)

In [ ]:
result = etc.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
prediction = Counter(np.equal(result, Y_test))
prediction[True] / (prediction[True] + prediction[False])

# 3. Bernoulli

In [ ]:
%%time

bnb = BernoulliNB()
bnb.fit(X_train, Y_train)

In [ ]:
result = bnb.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
prediction = Counter(np.equal(result, Y_test))
prediction[True] / (prediction[True] + prediction[False])

In [ ]:
confr3 = np.where(Y_test != result)

In [ ]:
confr3[0].shape

# 4. Decision Tree

In [ ]:
%%time

dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, Y_train)

In [ ]:
result = dtc.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
prediction = Counter(np.equal(result, Y_test))
prediction[True] / (prediction[True] + prediction[False])

# 5. GaussianNB

In [ ]:
%%time

gnb = GaussianNB()
gnb.fit(X_train, Y_train)

In [ ]:
result = gnb.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
confr5 = np.where(Y_test != result)

In [ ]:
confr5[0].shape

# 6. KNeighborsClassifier

In [ ]:
%%time

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_train, Y_train)

In [ ]:
result = neigh.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
neigh3 = KNeighborsClassifier(n_neighbors=3)
neigh3.fit(X_train, Y_train)

In [ ]:
result = neigh3.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
%%time

neigh5 = KNeighborsClassifier(n_neighbors=5)
neigh5.fit(X_train, Y_train)

In [ ]:
result = neigh5.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

# 7. Linear SVC

In [ ]:
svc = LinearSVC(random_state=0, tol=1e-5)
svc.fit(X_train, Y_train)

In [ ]:
result = svc.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

# 8. SVC

In [ ]:
scl = SVC()

In [ ]:
scl.fit(X_train, Y_train)

In [ ]:
result = scl.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

# 9. SGD Classifier

In [ ]:
sgd = SGDClassifier(max_iter=1000, tol=1e-3)

In [ ]:
sgd.fit(X_train, Y_train)

In [ ]:
result = sgd.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
sgd2 = SGDClassifier(loss='perceptron', penalty='l1', max_iter=1000, tol=1e-3)

In [ ]:
sgd2.fit(X_train, Y_train)

In [ ]:
result = sgd2.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

# 10. AdaBoostClassifier 

In [ ]:
adc = AdaBoostClassifier(DecisionTreeClassifier(random_state=0), n_estimators=100)

In [ ]:
adc.fit(X_train, Y_train)

In [ ]:
result = adc.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

# 11. BaggingClassifier

In [ ]:
bc = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=0), n_estimators=10, random_state=0)

In [ ]:
bc.fit(X_train, Y_train)

In [ ]:
result = bc.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
bc2 = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=5), n_estimators=10, random_state=0)

In [ ]:
bc2.fit(X_train, Y_train)

In [ ]:
result = bc2.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

# 12. XGB Classifier

In [ ]:
model = XGBClassifier(objective='multi:softmax')

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
result = model.predict(X_test)

In [ ]:
print(classification_report(np.vectorize(index_to_label.get)(Y_test), np.vectorize(index_to_label.get)(result)))

In [ ]:
prediction = Counter(np.equal(result, Y_test))
prediction[True] / (prediction[True] + prediction[False])